# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [21]:
# import libraries
import pandas as pd
import pickle

from sqlalchemy import create_engine
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.model_selection import GridSearchCV



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Carbonero\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Carbonero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Carbonero\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Carbonero\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data_table.db')
df = pd.read_sql_table('data_table',engine)
X = df[df.columns[1]].astype("string")
Y = df[df.columns[5:]].astype('int')

In [3]:
X.dtypes

string[python]

In [4]:
Y.head()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: string

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize text
    words = text.split()
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]

    return lemmed

print('La fase orginal es :',X.iloc[1])
tokens = tokenize(X.iloc[1])

print(' la frase modificada es: ',tokens)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#Classifier used: Random Forest Classifier. MultioutpurClassifier is needed since RFC only output one class for input

pipe= Pipeline([
        ('vect', CountVectorizer(analyzer = 'word', tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier(n_estimators=5)))
    ])

pipe


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000025C64ACBEE0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=5)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2, random_state=42)


In [9]:
# Fit the Random Forest Classifier 
pipe.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000025C64ACBEE0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=5)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipe.predict(X_test)

In [11]:
def clasification_metrics(y_test,y_pred):
    
    ''' 
    Function created to simplify the evaluation step
    It will give the f1 score, precision and recall for each feature or category
    
    INPUTS:
        y_test: true output for a certain text
        y_pred: Predicted output for the same text
        
    OUTPUT:
        accuracy_list: An array with the accurary score for each feature
        
        
    '''
    i = 0  
    accuracy_list = []
    for col in y_test:
        print('Feature:',col)
        print(classification_report(y_test[col], y_pred[:, i]))
        accuracy = accuracy_score(y_test[col], y_pred[:, i], normalize=True)
        accuracy_list.append(accuracy)
      
    
    return accuracy_list
        
        
        

In [12]:
accuary_listing = clasification_metrics(y_test,y_pred)


Feature: request
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      4320
           1       0.73      0.47      0.57       903

    accuracy                           0.88      5223
   macro avg       0.81      0.72      0.75      5223
weighted avg       0.87      0.88      0.87      5223

Feature: offer
              precision    recall  f1-score   support

           0       1.00      0.89      0.94      5201
           1       0.00      0.09      0.01        22

    accuracy                           0.88      5223
   macro avg       0.50      0.49      0.47      5223
weighted avg       0.99      0.88      0.94      5223

Feature: aid_related
              precision    recall  f1-score   support

           0       0.63      0.96      0.76      3037
           1       0.78      0.21      0.33      2186

    accuracy                           0.64      5223
   macro avg       0.70      0.58      0.54      5223
weighted avg       0.

C:\Users\Carbonero\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Carbonero\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Carbonero\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      4877
           1       0.06      0.10      0.07       346

    accuracy                           0.84      5223
   macro avg       0.50      0.49      0.49      5223
weighted avg       0.87      0.84      0.85      5223

Feature: transport
              precision    recall  f1-score   support

           0       0.95      0.89      0.92      4994
           1       0.03      0.08      0.05       229

    accuracy                           0.85      5223
   macro avg       0.49      0.49      0.48      5223
weighted avg       0.91      0.85      0.88      5223

Feature: buildings
              precision    recall  f1-score   support

           0       0.95      0.89      0.92      4962
           1       0.06      0.14      0.09       261

    accuracy                           0.85      5223
   macro avg       0.51      0.51      0.50      5223
weighted avg       0.91      0.85   

In [13]:
print(np.mean(accuary_listing))


0.8471376603484588


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = pipe.get_params()
parameters



{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000025C64ACBEE0>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=5)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000025C64ACBEE0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=5)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,


In [15]:
# After taking a look at this 
parameters = {
        'tfidf__use_idf': (True, False), #rather to use or not tdif 
        'clf__estimator__n_estimators': [5, 10] # Number of estimator ( trees) from 10 to 20, Now: 20
}
#Perform grid search
cv = GridSearchCV(pipe, param_grid = parameters)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# Fit the first tuned model
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000025C64ACBEE0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=5)))]),
             param_grid={'clf__estimator__n_estimators': [5, 10],
                         'tfidf__use_idf': (True, False)})

In [17]:
y_pred = cv.predict(X_test)



In [18]:
accuary_listing = clasification_metrics(y_test,y_pred)


Feature: request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      4320
           1       0.78      0.40      0.53       903

    accuracy                           0.88      5223
   macro avg       0.83      0.69      0.73      5223
weighted avg       0.87      0.88      0.86      5223

Feature: offer
              precision    recall  f1-score   support

           0       1.00      0.91      0.95      5201
           1       0.00      0.05      0.00        22

    accuracy                           0.91      5223
   macro avg       0.50      0.48      0.48      5223
weighted avg       0.99      0.91      0.95      5223

Feature: aid_related
              precision    recall  f1-score   support

           0       0.62      0.97      0.76      3037
           1       0.81      0.17      0.29      2186

    accuracy                           0.64      5223
   macro avg       0.72      0.57      0.52      5223
weighted avg       0.

C:\Users\Carbonero\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Carbonero\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Carbonero\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

Feature: death
              precision    recall  f1-score   support

           0       0.96      0.91      0.93      4998
           1       0.04      0.08      0.05       225

    accuracy                           0.87      5223
   macro avg       0.50      0.49      0.49      5223
weighted avg       0.92      0.87      0.89      5223

Feature: other_aid
              precision    recall  f1-score   support

           0       0.87      0.92      0.90      4503
           1       0.27      0.17      0.21       720

    accuracy                           0.82      5223
   macro avg       0.57      0.55      0.55      5223
weighted avg       0.79      0.82      0.80      5223

Feature: infrastructure_related
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      4877
           1       0.07      0.09      0.08       346

    accuracy                           0.86      5223
   macro avg       0.50      0.50      0.50      5223
weighted

In [19]:
print(np.mean(accuary_listing))

0.8649544596701404


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
# Create a pickle file for the model 
with open ('classifier_model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.